In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -U --q transformers datasets accelerate sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Import library

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import datasets
from sklearn.metrics import *

# Load data

In [5]:
sufix_path = "/content/drive/MyDrive/NLU_NCKH/notebook/Data/"
df_train = pd.read_csv(f"{sufix_path}Hotel Preprocessed/Train.csv")
df_dev = pd.read_csv(f"{sufix_path}Hotel Preprocessed/Dev.csv")
df_test = pd.read_csv(f"{sufix_path}Hotel Preprocessed/Test.csv")

print("Train: ", df_train.shape)
print("Dev: ",  df_dev.shape)
print("Test: ", df_test.shape )

Train:  (7180, 35)
Dev:  (795, 35)
Test:  (2030, 35)


# Preprocess data

In [6]:
import numpy as np

def toTextLabel(row):
    dict_label = {1: "negative", 2: "neutral", 3: "positive"}
    prompt = "Đánh giá thể hiện ý kiến "
    labels = [
        (aspect.replace("#", "_"), polarity)
        # f"{aspect.replace('&', '_').replace('#', '_')}-{dict_label[polarity]}"
        for aspect, polarity in zip(row.index, row.values) if polarity != 0
    ]
    for label in labels:
        prompt += f"[{dict_label[label[1]]}] cho [{label[0]}], "
    return prompt.lower()[:-2]

# df_train.drop("review", axis=1).apply(toTextLabel, axis=1).iloc[0]
df_train["Labels"] = df_train.drop("review", axis=1).apply(toTextLabel, axis=1)
df_test["Labels"] = df_test.drop("review", axis=1).apply(toTextLabel, axis=1)
df_dev["Labels"] = df_dev.drop("review", axis=1).apply(toTextLabel, axis=1)

In [7]:
label_lengths = df_train["Labels"].map(len)
review_lengths = df_train["review"].map(len)

# Tìm độ dài lớn nhất và chỉ số của hàng có độ dài lớn nhất
max_review_length = review_lengths.max()
max_review_index = review_lengths.idxmax()

max_labels_length = label_lengths.max()
max_labels_index = label_lengths.idxmax()

print(f"Max length review: {max_review_length}")
print(f"Index of max review: {max_review_index}")
print(f"Value of Review at max index: {df_train['Labels'][max_review_index]}")
print(f"Value of Review at max index: {df_train['review'][max_review_index]}")
print("-"*80)
print(f"Max length labels: {max_labels_length}")
print(f"Index of max length: {max_labels_index}")
print(f"Value of Labels at max index: {df_train['Labels'][max_labels_index]}")
print(f"Value of Labels at max index: {df_train['review'][max_labels_index]}")

Max length review: 814
Index of max review: 4060
Value of Review at max index: đánh giá thể hiện ý kiến [positive] cho [hotel_general]
Value of Review at max index: chúng_tôi đi team building tại cần giờ và đặt phòng tại resort này rất nhiều điểm cộng nhân_viên tiếp_tân rất thân_thiện lúc_nào cũng nở nụ cười giúp_đỡ khách rất ân_cần chu_đáo nhà_hàng carot khá đẹp nấu_ăn rất ngon nhân_viên thân_thiện niềm_nở tư_vấn menu rất nhiệt_tình hồ bơi đẹp khá sạch_sẽ vị_trí ngay bãi 304 gần chợ gần biển gần trạm xe_buýt thuận_tiện cho việc ăn_chơi nhảy_múa khá nhiều nhà_hàng quán ăn gần resort nấu thức_ăn cũng rất ngon hải_sản luôn_luôn tươi phòng superior khá nhỏ nhưng cũng đầy_đủ vật_dụng cần_thiết cho khách quang_cảnh vườn yên_tĩnh và mát_mẻ xíu xíu điểm trừ nhà_hàng carot nấu ngon nhưng giá_cả khá mắc phòng superior khá nhỏ lúc muốn tụ_tập đông người tại 1 phòng để chơi hơi chật phòng tắm cũng nhỏ không có bồn_tắm trong nhà_tắm tổng_quan resort 3 sao rất đáng để nghỉ_ngơi
--------------------

In [8]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import datasets
MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [9]:
max_len_input = max(len(tokenizer.encode(review)) for review in df_train['review']) + 13
max_len_target = len(tokenizer.encode(df_train['Labels'][max_labels_index]))
print(max_len_input)
print(max_len_target)

274
139


In [10]:
import random
import torch
from torch.utils.data import Dataset, DataLoader


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [11]:
from datasets import Dataset
dataset_train =  Dataset.from_pandas(df_train)
dataset_dev = Dataset.from_pandas(df_dev)
dataset_test = Dataset.from_pandas(df_test)

example = dataset_train[0]

print("Review:", example["review"])
print("Traget:", example["Labels"])

Review: vừa_qua tôi có dùng dịch_vụ tại khách_sạn tc hotel premium ngọc_lan ngọc_lan đà_lạt
Traget: đánh giá thể hiện ý kiến [positive] cho [hotel_general], [positive] cho [service_general]


In [12]:
dataset_train

Dataset({
    features: ['review', 'FACILITIES#CLEANLINESS', 'FACILITIES#COMFORT', 'FACILITIES#DESIGN&FEATURES', 'FACILITIES#GENERAL', 'FACILITIES#MISCELLANEOUS', 'FACILITIES#PRICES', 'FACILITIES#QUALITY', 'FOOD&DRINKS#MISCELLANEOUS', 'FOOD&DRINKS#PRICES', 'FOOD&DRINKS#QUALITY', 'FOOD&DRINKS#STYLE&OPTIONS', 'HOTEL#CLEANLINESS', 'HOTEL#COMFORT', 'HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL', 'HOTEL#MISCELLANEOUS', 'HOTEL#PRICES', 'HOTEL#QUALITY', 'LOCATION#GENERAL', 'ROOMS#CLEANLINESS', 'ROOMS#COMFORT', 'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 'ROOMS#MISCELLANEOUS', 'ROOMS#PRICES', 'ROOMS#QUALITY', 'ROOM_AMENITIES#CLEANLINESS', 'ROOM_AMENITIES#COMFORT', 'ROOM_AMENITIES#DESIGN&FEATURES', 'ROOM_AMENITIES#GENERAL', 'ROOM_AMENITIES#MISCELLANEOUS', 'ROOM_AMENITIES#PRICES', 'ROOM_AMENITIES#QUALITY', 'SERVICE#GENERAL', 'Labels'],
    num_rows: 7180
})

# Prepare input for model

In [13]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import datasets

# Initialize constants and tokenizer
# MAX_LEN_INPUT = 170
# MAX_LEN_TARGET = 70
# MODEL_NAME = "VietAI/vit5-base"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_examples(examples):
    prefix = "aspect-based sentiment analysis: "
    reviews = [prefix + review for review in examples['review']]
    labels = [label for label in examples['Labels']]

    # Tokenize the reviews
    model_inputs = tokenizer(
        reviews,
        max_length=max_len_input,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Tokenize the labels
    labels = tokenizer(
        labels,
        max_length=max_len_target,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Adjust labels for compatibility with PyTorch
    temp_labels = labels['input_ids']
    temp_labels[temp_labels == tokenizer.pad_token_id] = -100

    # Add target_ids and target_mask to model_inputs
    model_inputs['labels'] = temp_labels
    model_inputs['decoder_attention_mask'] = labels['attention_mask']

    return model_inputs


# Apply the preprocessing function to the datasets
dataset_train = dataset_train.map(preprocess_examples, batched=True)
dataset_dev = dataset_dev.map(preprocess_examples, batched=True)
dataset_test = dataset_test.map(preprocess_examples, batched=True)

# Set the format of the datasets to PyTorch tensors
dataset_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
dataset_dev.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
dataset_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

Map:   0%|          | 0/7180 [00:00<?, ? examples/s]

Map:   0%|          | 0/795 [00:00<?, ? examples/s]

Map:   0%|          | 0/2030 [00:00<?, ? examples/s]

# Define metrics

In [42]:
import re
import numpy as np
from sklearn.metrics import classification_report

true = ["đánh giá thể hiện ý kiến [positive] cho [restaurant#general], [negative] cho [ambience#general]", "đánh giá thể hiện ý kiến [positive] cho [restaurant#general], [negative] cho [ambience#general]"]
pred = ["đánh giá thể hiện ý kiến [positive] cho [restaurant#general], [neutral] cho [ambience#general]", "đánh giá thể hiện ý kiến [positive] cho [restaurant#general], [negative] cho [ambience#general]"]

aspects = df_train.drop(["review", "Labels"], axis=1).columns.tolist()
aspects = [aspect.lower() for aspect in aspects]
sentiments = {'negative': 1, 'neutral': 2, 'positive': 3}
replacements = {0: 'none', 1: 'negative', 2: 'neutral', 3: 'positive'}

def parse_output(batchs):
    r = "\[(.+?)\]"
    outputs = []
    for label in batchs:
        output = re.findall(r, label)
        output_list = [(output[i+1].replace(" ", "").replace("_","#"),
                        output[i]) for i in range(0, len(output), 2)]
        y = [0] * len(aspects)
        for aspect, sentiment in output_list:
            a_split = aspect.split("#")
            if a_split[0] == 'room':
              aspect = a_split[0] + "_" + a_split[1] + "#" + a_split[2]
              y[aspects.index(aspect)] = sentiments[sentiment]
            else:
              y[aspects.index(aspect)] = sentiments[sentiment]
        outputs.append(y)
    return outputs

def aspect_detection_eval(y_true, y_pred):
    aspect_test = []
    aspect_pred = []

    for row_test, row_pred in zip(y_true, y_pred):
        for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
            aspect_test.append(bool(col_test) * aspects[index])
            aspect_pred.append(bool(col_pred) * aspects[index])
    metrics = classification_report(aspect_test, aspect_pred, zero_division=1, digits=4)
    return metrics

def sentiment_classification_eval(y_true, y_pred):
    y_true_flat = np.array(y_true).flatten()
    y_pred_flat = np.array(y_pred).flatten()
    target_names = list(map(str, replacements.values()))
    metrics = classification_report(y_true_flat, y_pred_flat, zero_division=1, target_names=target_names, digits=4)
    return metrics

def combination_eval(y_true, y_pred):
    aspect_polarity_true = []
    aspect_polarity_pred = []

    for row_test, row_pred in zip(y_true, y_pred):
        for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
            aspect_polarity_true.append(f'{aspects[index]},{replacements[col_test]}')
            aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')
    metrics = classification_report(aspect_polarity_true, aspect_polarity_pred, zero_division=1, digits=4)
    return metrics

# # Gọi hàm parse_output với true và pred
# y_true = parse_output(true)
# y_pred = parse_output(pred)
# print(list(map(str, replacements.values())))
# # Đánh giá
# print("## Aspect Detection Evaluate ##")
# metrics = aspect_detection_eval(y_true, y_pred)
# print(metrics)

# print("\n## Sentiment Classification Evaluate ##")
# metrics = sentiment_classification_eval(y_true, y_pred)
# print(metrics)

# print("\n## Combination Evaluate (Aspect + Polarity detection) ##")
# metrics = combination_eval(y_true, y_pred)
# print(metrics)


In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)
!nvidia-smi

Device: cuda
Mon Jul 15 05:50:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+------------

# Training model

In [17]:
import wandb
wandb.init(mode='disabled')

In [18]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
# model.to(device)
# Define training arguments
training_args = TrainingArguments(
    './vit5-absa-axl',
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy='no'
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_dev,
    # compute_metrics=compute_metrics
)

# Train the model
trainer.train()

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.342800,0.104200
2,0.092100,0.081770
3,0.072800,0.077803
4,0.060800,0.076820


TrainOutput(global_step=3592, training_loss=0.11670814497699185, metrics={'train_runtime': 2924.2027, 'train_samples_per_second': 9.821, 'train_steps_per_second': 1.228, 'total_flos': 9359492478566400.0, 'train_loss': 0.11670814497699185, 'epoch': 4.0})

# Load model

In [19]:
save = "/content/drive/MyDrive/NLU_NCKH/notebook/Checkpoints/vit5-absa-axl"
trainer.save_model(save)

In [20]:
import gc
del model, tokenizer,
torch.cuda.empty_cache()
gc.collect()

174

In [21]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(save).to(device)

# Prepare input to inference

In [22]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.labels)

# Inference and Evaluate

In [23]:
from torch.utils.data import DataLoader
from tqdm import tqdm
# Evaluate the model
# results = trainer.evaluate()
# Generate predictions
# MAX_LEN_INPUT = 170
# MAX_LEN_TARGET = 70

def generate_predictions(dataset):
    prefix = "aspect-based sentiment analysis: "
    input_texts = [prefix + review for review in dataset['review']]
    y_test = dataset['Labels']
    # tokenizer.add_tokens(["#", "&"])
    # Tokenize the reviews
    inputs = tokenizer(
        input_texts,
        max_length=max_len_input,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    encoding_data = customDataset(inputs, y_test)
    data_loader = DataLoader(encoding_data, batch_size=32, shuffle=False, num_workers=0)
    y_preds = []

    model.eval()
    for index, data in enumerate(tqdm(data_loader, desc="Predicting")):
      outputs = model.generate(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
          max_length=max_len_target,
          num_beams=4,
          early_stopping=True
      )
      generate = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
      y_preds.extend(generate)

    return y_test, y_preds

dataset_test = datasets.Dataset.from_pandas(df_test)

y_test, y_preds = generate_predictions(dataset_test)

Predicting: 100%|██████████| 64/64 [06:29<00:00,  6.09s/it]


In [47]:
for y, p in zip(y_test, y_preds):
  print(y)
  print(p)
  print("-"*80)

đánh giá thể hiện ý kiến [positive] cho [service_general]
đánh giá thể hiện ý kiến [positive] cho [service_general]
--------------------------------------------------------------------------------
đánh giá thể hiện ý kiến [neutral] cho [rooms_design&features]
đánh giá thể hiện ý kiến [positive] cho [rooms_design&features]
--------------------------------------------------------------------------------
đánh giá thể hiện ý kiến [positive] cho [rooms_cleanliness], [positive] cho [rooms_design&features]
đánh giá thể hiện ý kiến [positive] cho [rooms_cleanliness], [positive] cho [room_amenities_general]
--------------------------------------------------------------------------------
đánh giá thể hiện ý kiến [positive] cho [hotel_general]
đánh giá thể hiện ý kiến [positive] cho [hotel_general]
--------------------------------------------------------------------------------
đánh giá thể hiện ý kiến [positive] cho [food&drinks_quality]
đánh giá thể hiện ý kiến [positive] cho [food&drinks_quali

In [43]:
y_true = parse_output(y_test)
y_pred = parse_output(y_preds)

print("## Aspect Detection Evaluate ##")
report = aspect_detection_eval(y_true, y_pred)
print(report)

print("\n## Sentiment Classification Evaluate ##")
report = sentiment_classification_eval(y_true, y_pred)
print(report)

print("\n## Combination Evaluate (Aspect + Polarity detection) ##")
report = combination_eval(y_true, y_pred)
print(report)

## Aspect Detection Evaluate ##
                                precision    recall  f1-score   support

                                   0.9886    0.9904    0.9895     65760
        facilities#cleanliness     0.7778    0.8750    0.8235        16
            facilities#comfort     1.0000    0.0000    0.0000         6
    facilities#design&features     0.5556    0.5882    0.5714        17
            facilities#general     0.6364    0.3111    0.4179        45
      facilities#miscellaneous     0.3636    0.3636    0.3636        11
             facilities#prices     0.8000    0.6667    0.7273         6
            facilities#quality     0.5833    0.6364    0.6087        22
     food&drinks#miscellaneous     0.3333    0.1111    0.1667         9
            food&drinks#prices     0.5714    0.5000    0.5333         8
           food&drinks#quality     0.8614    0.8529    0.8571       102
     food&drinks#style&options     0.8571    0.8108    0.8333        74
             hotel#cleanliness 

In [ ]:
save = pd.DataFrame({
    'y_test': y_true,
    'y_pred': y_pred
})
save.to_csv("Auxiliary.csv")